In [5]:
%pip install Faker

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple


In [7]:
# Hằng số cho việc gán ID khách hàng
PROBABILITY_NEW_NORMAL_CUSTOMER = 0.6


class SyntheticSalesDataGenerator:
    """
    Một lớp để tạo sinh dữ liệu bán hàng tổng hợp cho một chuỗi siêu thị
    giả định tại Việt Nam, dựa trên một file dữ liệu gốc.

    Cập nhật: Đảm bảo mỗi SKU có một cặp Giá vốn và Giá bán nhất quán.
    """

    def __init__(self, source_csv_path: str, random_state: int = 42):
        """
        Khởi tạo generator.

        Args:
            source_csv_path (str): Đường dẫn đến file CSV gốc.
            random_state (int): Hạt giống cho các bộ tạo số ngẫu nhiên
                                để đảm bảo kết quả có thể tái lập.
        """
        self.rng = np.random.default_rng(random_state)
        self.py_random = random.Random(random_state)

        try:
            self.source_df = pd.read_csv(source_csv_path)
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file tại '{source_csv_path}'")
            raise

        # Cấu hình cho siêu thị ABC tại Việt Nam
        self.vietnam_branches = {
            "Hà Nội": ["ABC Cầu Giấy", "ABC Hoàn Kiếm", "ABC Thanh Xuân"],
            "TP.HCM": [
                "ABC Quận 1", "ABC Quận 7", "ABC Gò Vấp", "ABC Thủ Đức"
            ],
            "Đà Nẵng": ["ABC Hải Châu", "ABC Sơn Trà"],
        }
        self.all_branches = [
            branch for city_branches in self.vietnam_branches.values()
            for branch in city_branches
        ]
        self.branch_to_city = {
            branch: city for city, branches in self.vietnam_branches.items()
            for branch in branches
        }

        self.sku_price_map = self._create_sku_price_map()
        print(
            f"Đã tạo bảng giá nhất quán cho {len(self.sku_price_map)} "
            f"SKU duy nhất."
        )

        # Khởi tạo Faker để tạo dữ liệu giả
        self.faker = Faker('vi_VN')

    def _create_sku_price_map(self):
        """
        Tạo một từ điển để ánh xạ mỗi SKU tới một giá vốn và giá bán cố định,
        đảm bảo tính nhất quán của giá sản phẩm.
        """
        price_map = {}
        unique_sku_df = self.source_df.drop_duplicates(subset=['SKU'])

        for _, row in unique_sku_df.iterrows():
            sku = row['SKU']
            base_unit_cost = row['Unit Cost']
            base_sales_price = row['Sales Price']

            # Tạo giá mới với một chút nhiễu ngẫu nhiên (ổn định hơn)
            noise = self.rng.normal(1, 0.05)
            new_unit_cost = base_unit_cost * noise
            new_sales_price = base_sales_price * noise

            # Đảm bảo giá bán luôn cao hơn giá vốn ít nhất 8%
            min_sales = new_unit_cost * 1.08
            if new_sales_price < min_sales:
                uplift = float(self.rng.uniform(1.15, 1.40))
                new_sales_price = max(min_sales, new_unit_cost * uplift)

            price_map[sku] = {
                'unit_cost': round(new_unit_cost, 2),
                'sales_price': round(new_sales_price, 2),
            }
        return price_map

    def _promo_calendar(self) -> List[Tuple[datetime, datetime, float]]:
        """Tạo lịch khuyến mãi cho các năm."""
        promos: List[Tuple[datetime, datetime, float]] = []
        for y in [2024, 2025]:
            promos.append((datetime(y, 2, 1), datetime(y, 2, 10), 0.35))
            promos.append((datetime(y, 4, 27), datetime(y, 5, 2), 0.25))
            promos.append((datetime(y, 8, 30), datetime(y, 9, 3), 0.22))
            d = self._last_friday_of_november(y)
            promos.append((d, d + timedelta(days=2), 0.30))
        return promos

    @staticmethod
    def _last_friday_of_november(year: int) -> datetime:
        """Tìm ngày thứ Sáu cuối cùng của tháng 11 trong một năm."""
        d = datetime(year, 11, 30)
        while d.weekday() != 4:  # 4 là thứ Sáu (Monday=0)
            d -= timedelta(days=1)
        return d

    def _random_sale_time(self, start: datetime, end: datetime) -> datetime:
        """Tạo thời gian giao dịch ngẫu nhiên với các thiên vị nhất định."""
        total_seconds = int((end - start).total_seconds())
        t = start + timedelta(seconds=self.py_random.randrange(max(1, total_seconds)))

        # Giờ mở cửa 8-22h, ưu tiên khung giờ 17-20h
        t = t.replace(
            hour=self.py_random.randrange(8, 22),
            minute=self.py_random.randrange(0, 60),
            second=0
        )
        if self.py_random.random() < 0.5:
            t = t.replace(hour=self.py_random.choice([17, 18, 19, 20]))

        # Tăng xác suất rơi vào T7/CN
        if self.py_random.random() < 0.4 and t.weekday() not in (5, 6):
            t += timedelta(days=(5 - t.weekday()) % 7)  # Đẩy tới thứ 7
        return t

    def _dirty_data(self, df: pd.DataFrame, outlier_prob=0.02) -> pd.DataFrame:
        """Hàm nội bộ để làm 'dơ' dữ liệu một cách có kiểm soát."""
        print("Bắt đầu quá trình làm 'dơ' dữ liệu...")
        # 1. Tạo Outliers cho Số lượng
        outlier_indices_qty = df.sample(frac=outlier_prob).index
        df.loc[outlier_indices_qty, 'Số lượng'] *= np.random.randint(10, 20)
        print(
            f" -> Đã tạo outliers cho {len(outlier_indices_qty)} dòng "
            f"trong cột 'Số lượng'."
        )

        # 2. Tạo giá trị thiếu (NaN)
        for col in ['Discount', 'Nhóm khách hàng', 'Giới tính']:
            missing_prob = np.random.uniform(0.01, 0.15)
            missing_indices = df.sample(frac=missing_prob).index
            df.loc[missing_indices, col] = np.nan
            print(
                f" -> Đã tạo {len(missing_indices)} giá trị thiếu "
                f"trong cột '{col}'."
            )

        # 3. Tạo dòng trùng lặp (1–3%)
        dup_frac = float(self.rng.uniform(0.01, 0.03))
        if dup_frac > 0:
            dup_rows = df.sample(frac=dup_frac, random_state=self.rng)
            df = pd.concat([df, dup_rows], ignore_index=True)
            print(f" -> Đã nhân bản {len(dup_rows)} dòng ({dup_frac:.2%}).")

        # 4. Tạo giá trị không nhất quán
        # Lỗi thang đo đơn vị
        price_cols = [
            'Giá vốn đơn vị', 'Giá bán', 'COGS', 'Doanh thu ròng', 'Lợi nhuận'
        ]
        scale_frac = float(self.rng.uniform(0.005, 0.01))
        if scale_frac > 0 and not df.empty:
            sidx = df.sample(frac=scale_frac, random_state=self.rng).index
            factors = self.rng.choice([0.001, 1000.0], size=len(sidx))
            for c in price_cols:
                if c in df.columns:
                    df.loc[sidx, c] *= factors
            print(f" -> Lỗi quy đổi đơn vị giá: {len(sidx)} dòng.")

        # Phương thức thanh toán lạ
        bad_payment_frac = 0.005
        if 'Phương thức thanh toán' in df.columns and bad_payment_frac > 0:
            midx = df.sample(frac=bad_payment_frac, random_state=self.rng).index
            bad_vals = ["QR", "Cashh", "Creditt", None, ""]
            repl = self.py_random.choices(bad_vals, k=len(midx))
            df.loc[midx, 'Phương thức thanh toán'] = repl
            print(f" -> Phương thức thanh toán lạ: {len(midx)} dòng.")

        return df

    def generate_data(self, num_rows: int = 1000) -> pd.DataFrame:
        """
        Tạo sinh dữ liệu bán hàng tổng hợp.

        Args:
            num_rows (int): Số lượng dòng dữ liệu cần tạo.

        Returns:
            pd.DataFrame: DataFrame chứa dữ liệu tổng hợp đã được làm "dơ".
        """
        print(f"Bắt đầu tạo sinh {num_rows} dòng dữ liệu mới...")
        new_data = []

        unique_customers = self.source_df['Customer Type'].dropna().unique()
        unique_genders = self.source_df['Gender'].dropna().unique()
        unique_payment = self.source_df['Payment Method'].dropna().unique()
        promo_calendar = self._promo_calendar()
        source_records = self.source_df.to_dict('records')

        start_date = datetime(2024, 1, 1)
        end_date = datetime.now()

        def draw_discount(ts: datetime) -> float:
            base = float(self.rng.uniform(0.00, 0.10))
            for s, e, boost in promo_calendar:
                if s <= ts <= e:
                    return round(min(boost, base + float(self.rng.uniform(0.05, boost))), 4)
            return 0.0 if self.rng.random() < 0.2 else round(base, 4)

        for _ in range(num_rows):
            base_row = self.py_random.choice(source_records)
            sku = base_row['SKU']

            prices = self.sku_price_map[sku]
            unit_cost = prices['unit_cost']
            sales_price = prices['sales_price']

            quantity = int(self.rng.integers(1, 12))
            cogs = unit_cost * quantity
            gross_revenue = sales_price * quantity

            random_date = self._random_sale_time(start_date, end_date)
            disc_rate = draw_discount(random_date)
            discount_value = gross_revenue * disc_rate
            revenue = gross_revenue - discount_value
            profit = revenue - cogs

            branch = self.py_random.choice(self.all_branches)
            city = self.branch_to_city[branch]

            new_row = {
                'Thời gian': random_date,
                'Thành phố': city,
                'Chi nhánh': branch,
                'Nhóm khách hàng': self.py_random.choice(unique_customers),
                'Giới tính': self.py_random.choice(unique_genders),
                'Phân loại': base_row['Product Category'],
                'Tên SP': base_row['Product'],
                'SKU': sku,
                'Giá vốn đơn vị': unit_cost,
                'Giá bán': sales_price,
                'Số lượng': quantity,
                'COGS': round(cogs, 2),
                'Discount': round(discount_value, 2),
                'Phương thức thanh toán': self.py_random.choice(unique_payment),
                'Doanh thu ròng': round(revenue, 2),
                'Lợi nhuận': round(profit, 2),
            }
            new_data.append(new_row)

        synthetic_df = pd.DataFrame(new_data)
        synthetic_df = synthetic_df.sort_values("Thời gian").reset_index(drop=True)

        synthetic_df = self._dirty_data(synthetic_df)

        print("Hoàn tất tạo sinh và làm 'dơ' dữ liệu.")
        return synthetic_df


def assign_customer_ids(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gán ID cho mỗi giao dịch trong DataFrame.

    Logic:
    - Giao dịch có thông tin khách hàng bị thiếu ('Nhóm khách hàng', 'Giới tính')
      sẽ được gán một ID 'CUS-xxxx' duy nhất, không tái sử dụng (coi như
      khách vãng lai ẩn danh).
    - Khách hàng 'Member' có xu hướng quay lại (tái sử dụng ID).
    - Khách hàng 'Normal' chủ yếu là khách mới (tạo ID mới).
    - Một khách hàng không thể có hai giao dịch trong cùng một ngày.

    Args:
        df (pd.DataFrame): DataFrame đầu vào chứa dữ liệu giao dịch.

    Returns:
        pd.DataFrame: DataFrame với cột 'ID' đã được thêm vào.
    """
    print("Bắt đầu quá trình gán ID...")
    df['ID'] = None
    customer_profiles = {}  # {profile_key: [id1, id2]}
    daily_transactions = {}  # {(cust_id, date): True}
    
    # Bộ đếm dùng chung để đảm bảo mọi ID 'CUS-' là duy nhất
    customer_id_counter = 1
    
    # Biến đếm để báo cáo ở cuối
    num_anonymous_assigned = 0

    def create_new_id_for_profile(profile_key, transaction_date):
        """
        Hàm trợ giúp để tạo và đăng ký ID cho khách hàng CÓ HỒ SƠ.
        """
        nonlocal customer_id_counter
        new_id = f"CUS-{customer_id_counter:04d}"
        customer_id_counter += 1

        if profile_key not in customer_profiles:
            customer_profiles[profile_key] = []
        customer_profiles[profile_key].append(new_id)

        daily_transactions[(new_id, transaction_date)] = True
        return new_id

    for index, row in df.iterrows():
        # Xử lý các dòng thiếu thông tin ràng buộc 
        if pd.isna(row['Nhóm khách hàng']) or pd.isna(row['Giới tính']):
            # Gán một ID CUS- duy nhất và không lưu lại hồ sơ
            assigned_id = f"CUS-{customer_id_counter:04d}"
            customer_id_counter += 1
            df.loc[index, 'ID'] = assigned_id
            num_anonymous_assigned += 1
            # Chuyển sang dòng tiếp theo
            continue

        # Xử lý các dòng có đủ thông tin 
        profile_key = (row['Thành phố'], row['Nhóm khách hàng'], row['Giới tính'])
        transaction_date = row['Thời gian'].date()
        assigned_id = None

        # Logic cho khách hàng 'Member'
        if row['Nhóm khách hàng'] == 'Member':
            if profile_key in customer_profiles:
                existing_ids = customer_profiles[profile_key]
                random.shuffle(existing_ids)
                for cust_id in existing_ids:
                    if (cust_id, transaction_date) not in daily_transactions:
                        assigned_id = cust_id
                        daily_transactions[(assigned_id, transaction_date)] = True
                        break
            if assigned_id is None:
                assigned_id = create_new_id_for_profile(profile_key, transaction_date)

        # Logic cho khách hàng 'Normal'
        elif row['Nhóm khách hàng'] == 'Normal':
            use_new_id = (
                random.random() < PROBABILITY_NEW_NORMAL_CUSTOMER
                or profile_key not in customer_profiles
            )
            if use_new_id:
                assigned_id = create_new_id_for_profile(profile_key, transaction_date)
            else:
                existing_ids = customer_profiles[profile_key]
                random.shuffle(existing_ids)
                for cust_id in existing_ids:
                    if (cust_id, transaction_date) not in daily_transactions:
                        assigned_id = cust_id
                        daily_transactions[(assigned_id, transaction_date)] = True
                        break
                if assigned_id is None:
                    assigned_id = create_new_id_for_profile(profile_key, transaction_date)

        df.loc[index, 'ID'] = assigned_id

    # Sắp xếp lại cột để ID lên đầu
    cols = ['ID'] + [c for c in df.columns if c != 'ID']
    df = df[cols]
    
    num_total_assigned = df['ID'].notna().sum()
    num_profile_based = num_total_assigned - num_anonymous_assigned

    print("Hoàn tất gán ID.")
    print(f" -> Đã gán {num_profile_based} ID cho khách hàng có hồ sơ (Member/Normal).")
    print(f" -> Đã gán {num_anonymous_assigned} ID duy nhất cho các giao dịch thiếu thông tin.")
    
    return df

In [8]:
if __name__ == "__main__":
    SOURCE_FILE_PATH = r'../data/Total-Supermarket-Sales.csv'
    OUTPUT_FILE_PATH = r'Supermarket_Sales_generated.xlsx' # File path
    NUM_ROWS_TO_GENERATE = 7000

    # Khởi tạo generator
    generator = SyntheticSalesDataGenerator(source_csv_path=SOURCE_FILE_PATH)

    # Tạo dữ liệu
    synthetic_sales_df = generator.generate_data(num_rows=NUM_ROWS_TO_GENERATE)
    final_df = assign_customer_ids(synthetic_sales_df)
    try:
        final_df.to_excel(OUTPUT_FILE_PATH, index=False)
        print(f"\nĐã lưu thành công dữ liệu cuối cùng vào file: {OUTPUT_FILE_PATH}")
    except Exception as e:
        print(f"\nLỗi khi lưu file Excel: {e}")

Đã tạo bảng giá nhất quán cho 59 SKU duy nhất.
Bắt đầu tạo sinh 7000 dòng dữ liệu mới...
Bắt đầu quá trình làm 'dơ' dữ liệu...
 -> Đã tạo outliers cho 140 dòng trong cột 'Số lượng'.
 -> Đã tạo 332 giá trị thiếu trong cột 'Discount'.
 -> Đã tạo 355 giá trị thiếu trong cột 'Nhóm khách hàng'.
 -> Đã tạo 940 giá trị thiếu trong cột 'Giới tính'.
 -> Đã nhân bản 183 dòng (2.62%).
 -> Lỗi quy đổi đơn vị giá: 59 dòng.
 -> Phương thức thanh toán lạ: 36 dòng.
Hoàn tất tạo sinh và làm 'dơ' dữ liệu.
Bắt đầu quá trình gán ID...
Hoàn tất gán ID.
 -> Đã gán 5895 ID cho khách hàng có hồ sơ (Member/Normal).
 -> Đã gán 1288 ID duy nhất cho các giao dịch thiếu thông tin.

Đã lưu thành công dữ liệu cuối cùng vào file: Supermarket_Sales_generated.xlsx


In [9]:
final_df

,ID,Thời gian,Thành phố,Chi nhánh,Nhóm khách hàng,Giới tính,Phân loại,Tên SP,SKU,Giá vốn đơn vị,Giá bán,Số lượng,COGS,Discount,Phương thức thanh toán,Doanh thu ròng,Lợi nhuận
0,CUS-0001,2024-01-01 08:21:00,Hà Nội,ABC Thanh Xuân,Member,Female,Electronics,Tablet,1002,17955.07,23341.59,1,17955.07,0.00,E-Wallet,23341.59,5386.52
1,CUS-0002,2024-01-01 09:16:00,TP.HCM,ABC Quận 1,Normal,Male,Electronics,Tablet,1002,17955.07,23341.59,7,125685.49,0.00,Cash,163391.13,37705.64
2,CUS-0003,2024-01-01 10:38:00,TP.HCM,ABC Quận 1,Normal,Male,Electronics,Bluetooth Speaker,1005,2094.52,2722.88,7,14661.64,306.87,Cash,18753.29,4091.65
3,CUS-0004,2024-01-01 16:31:00,TP.HCM,ABC Gò Vấp,Member,Male,Electronics,USB Cable,1008,244.65,318.04,6,1467.90,11.45,E-Wallet,1896.79,428.89
4,CUS-0005,2024-01-01 17:11:00,Hà Nội,ABC Hoàn Kiếm,Normal,Female,Groceries,Soft Drink (500ml),1030,40.73,52.95,4,162.92,14.91,Credit Card,196.89,33.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7178,CUS-0266,2025-08-22 12:45:00,Hà Nội,ABC Cầu Giấy,Member,Male,Groceries,Milk,1021,57.56,74.83,84,402.92,4.71,Cash,519.10,116.18
7179,CUS-3078,2024-04-24 13:54:00,Đà Nẵng,ABC Sơn Trà,Normal,Male,Personal Care,Shampoo,1051,186.75,242.78,1,186.75,19.93,Cash,222.85,36.10
7180,CUS-3079,2024-03-03 19:16:00,TP.HCM,ABC Quận 7,Member,NaN,Groceries,Cooking Oil,1028,143.27,186.25,4,573.08,0.00,Credit Card,745.00,171.92
7181,CUS-3080,2025-01-18 18:20:00,Hà Nội,ABC Thanh Xuân,Normal,Male,Household Items,Laundry Basket,1050,434.68,565.08,7,3042.76,NaN,Credit Card,3955.16,912.40
